In [75]:
import math

def choose(n, r, k):
    return math.factorial(n)//math.factorial(r)//math.factorial(k)//math.factorial(n-r-k)

epsilon = 0.1
MAX = 4
#alpha = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

playercards = [0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0]
alpha = [1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1]

In [76]:
def probTwocards(p1, p2, count1, count2, total): #p1: probability of first card, p2: probability of second, count1: number you want of first, count2: number you want of second, total: total num of cards
    max = MAX*2
    if count1+count2>max or count1+count2>total:
        return 0
    #elif total > max:
        #return choose(max, count1, count2)* (p1)**count1*(p2)**count2*(1-(p1+p2))**(max-count1-count2)
    else:
        return choose(total, count1, count2)* (p1)**count1*(p2)**count2*(1-(p1+p2))**(total-count1-count2)

    
def probOnecard(p, count, total):  #p: probability, count: number you want, total: total number of cards
    max = MAX
    if count>max or count>total:
        return 0
    #elif num > max:
        #return choose(max, count, 0)* (p)**count*(1-(p))**(max-count)
    else:
        return choose(total, count, 0)* (p)**count*(1-(p))**(total-count)

def twotype(count1, count2, card1, card2, num): #count: number you want, card: card index, num: number of cards
    p1 = (alpha[card1] + 1)/(sum(alpha)+1)
    p2 = (alpha[card2] + 1)/(sum(alpha)+1) 
    prob = 0
    for i in range(max(count1-playercards[card1], 0), 5-playercards[card1]):
        for j in range(max(count2-playercards[card2], 0), 5-playercards[card2]):
            prob = prob + probTwocards(p1, p2, i, j, num)
    
    return prob

def onetype(count, card, num):
    p = (alpha[card] + 1)/(sum(alpha)+1)
    prob=0
    for i in range(max(count-playercards[card], 0), 5-playercards[card]):
        prob = prob+probOnecard(p, i, num)
    return min(prob, 1)

def straight(card, num):
    p = (alpha[card] + 1)/(sum(alpha)+1)
    prob = 1
    for i in range(card, card+4):
        prob = prob*probOnecard(p, 1, num)
    return prob

In [84]:
twotypeprob = [[[0 for x in range(13)] for y in range(13)] for z in range(1, 4)]
onetypeprob = [[0 for x in range(13)] for y in range(1, 5)] 

def updateTwoType():
    
    largest = twotype(2, 2, 0, 1, 8)
    ind = [0, 0, 0]
    for k in range(0, 3):
        for i in range(0, 13):
            for j in range(0, 13):
                l = twotype(k+2, max(k+1, 2), i, j, 8)
                twotypeprob[k][i][j] = l
                if(l> largest):
                    largest = l
                    ind = [k, i, j]
                    
def updateOneType():
    largestone = onetype(1, 0, 8)
    indone = [0, 0]
    for k in range(0, 4):
        for i in range(0, 13):
            l = onetype(k, i, 8)
            onetypeprob[k][i] = l
            if(l> largestone):
                largestone = l
                indone = [k, i]
            
            

In [78]:
twotype(3, 2, 5, 6, 8)

0.3445628047265626

In [79]:
playercards[5]

3

In [83]:
probTwocards(0.2, 0.2, 0, 0, 8)
    

0.016796159999999994

In [15]:
twotype(3, 2, 1, 2, 6, alpha)

0.0038820301783264737

In [16]:
twotype(3, 2, 1, 2, 14, alpha)

0.02230568510897728

In [135]:
probOnecard(0.5, 1, 10)+probOnecard(0.5, 2, 10)+probOnecard(0.5, 3, 10)+probOnecard(0.5, 4, 10)

0.9375

In [16]:
onetype(2, 10, 10)

0.2622661335000001